In [1]:
import numpy as np
import pandas as pd

In [2]:
def generate_request_time_linear(ts_min, ts_max, work_time):
    T_server = [0]
    while True:
        new_t = T_server[-1] + (ts_max - ts_min) * np.random.rand() + ts_min
        if new_t > work_time:
            break
        T_server.append(new_t)
    T_server.pop(0)
    return np.asarray(T_server)

def generate_process_time_linear(tz_min, tz_max, n):
    return (tz_max - tz_min) * np.random.rand(n) + tz_min

def exp_func(lmbd, p):
    return -1 / lmbd * np.log(p)

def generate_request_time_exp(lmbd, work_time):
    T_server = [0]
    while True:
        new_t = T_server[-1] + exp_func(lmbd, np.random.rand())
        if new_t > work_time:
            break
        T_server.append(new_t)
    T_server.pop(0)
    return np.asarray(T_server)

def generate_process_time_exp(lmbd, n):
    return exp_func(lmbd, np.random.rand(n))

def get_f_o_t(cur_t, last_start_s1, last_end_s1, last_start_s2, last_end_s2):
    last_t = max(last_start_s1, last_start_s2)
    two = max(0, (min(last_end_s1, last_end_s2) - last_t))
    free = max(0, cur_t - max(last_end_s1, last_end_s2))
    one = cur_t - last_t - two - free
    return (free, one, two)

def simulate(Ts, Tz, work_time):
    last_start_s1 = 0
    last_end_s1 = 0
    last_start_s2 = 0
    last_end_s2 = 0
    processed_signals = 0

    free_time = 0
    one_work_time = 0
    two_work_time = 0

    for i in range(len(Ts)):
        if Ts[i] + Tz[i] <= work_time:
            if last_end_s1 <= Ts[i]:
                free, one, two = get_f_o_t(Ts[i], last_start_s1, last_end_s1, last_start_s2, last_end_s2)
                free_time += free
                one_work_time += one
                two_work_time += two

                last_start_s1, last_end_s1 = Ts[i], Ts[i] + Tz[i]
                processed_signals += 1
            elif last_end_s2 <= Ts[i]:
                free, one, two = get_f_o_t(Ts[i], last_start_s1, last_end_s1, last_start_s2, last_end_s2)
                free_time += free
                one_work_time += one
                two_work_time += two

                last_start_s2, last_end_s2 = Ts[i], Ts[i] + Tz[i]
                processed_signals += 1
    
    free, one, two = get_f_o_t(work_time, last_start_s1, last_end_s1, last_start_s2, last_end_s2)
    free_time += free
    one_work_time += one
    two_work_time += two

    return (processed_signals, (free_time, one_work_time, two_work_time))

In [8]:
work_time = 1 * 60 * 60

# linear
tz_min = 1 / 2
tz_max = 5 / 6

ts_min = 1
ts_max = 5

lmbd_z_lin = 2 / (tz_max + tz_min)
lmbd_s_lin = 2 / (ts_max + ts_min)

# exp
lmbd_z_exp = 1.5
ts_exp = 2

lmbd_s_exp = 1 / ts_exp

In [9]:
T_req = generate_request_time_linear(tz_min, tz_max, work_time)
T_processing = generate_process_time_linear(ts_min, ts_max, len(T_req))

processed_requests, fot_times = simulate(T_req, T_processing, work_time)

p0 = fot_times[0] / work_time
p1 = fot_times[1] / work_time
p2 = fot_times[2] / work_time

Q = processed_requests / len(T_req)
S = processed_requests / work_time
k = (fot_times[1] * 1 + fot_times[2] * 2) / work_time

print(p0)
print(p1)
print(p2)
print('========')
print(Q)
print(S)
print(k)


0.012804308296957317
0.21091545799796796
0.7762802337050747
0.3954823180892427
0.5933333333333334
1.7634759254081176


In [ ]:
# Linear (test)
test_p = lmbd_z_lin / lmbd_s_lin

p0 = 1 / (1 + test_p + test_p**2 / 2)
p1 = test_p * p0
p2 = test_p**2 / 2 * p0

Q = 1 - p2
S = lmbd_z_lin * Q
# k = test_p * Q
k = S / lmbd_s_lin

print(p0)
print(p1)
print(p2)
print('========')
print(Q)
print(S)
print(k)

0.064
0.28800000000000003
0.648
0.352
0.5279999999999999
1.5839999999999999


In [11]:
T_req = generate_request_time_exp(lmbd_z_exp, work_time)
T_processing = generate_process_time_exp(lmbd_s_exp, len(T_req))

processed_requests, fot_times = simulate(T_req, T_processing, work_time)

p0 = fot_times[0] / work_time
p1 = fot_times[1] / work_time
p2 = fot_times[2] / work_time

Q = processed_requests / len(T_req)
S = processed_requests / work_time
k = (fot_times[1] * 1 + fot_times[2] * 2) / work_time

print(p0)
print(p1)
print(p2)
print('========')
print(Q)
print(S)
print(k)


0.11008548250203995
0.3493755671210354
0.5405389503769247
0.4685212298682284
0.7111111111111111
1.4304534678748848


In [ ]:
# Exp (test)
test_p = lmbd_z_exp / lmbd_s_exp

p0 = 1 / (1 + test_p + test_p**2 / 2)
p1 = test_p * p0
p2 = test_p**2 / 2 * p0

Q = 1 - p2
S = lmbd_z_exp * Q
# k = test_p * Q
k = S / lmbd_s_exp

print(p0)
print(p1)
print(p2)
print('========')
print(Q)
print(S)
print(k)

0.11764705882352941
0.3529411764705882
0.5294117647058824
0.47058823529411764
0.7058823529411764
1.4117647058823528


In [15]:
rows = 10
table_cols = ['p0', 'p1', 'p2', 'Q', 'S', 'k']
indexes = list(map(str, range(1, rows + 1)))
indexes = ['theor'] + indexes + ['mu', 'sigma']

In [16]:
table_vals = np.zeros((rows + 3, 6))

test_p = lmbd_z_lin / lmbd_s_lin
p0 = 1 / (1 + test_p + test_p**2 / 2)
p1 = test_p * p0
p2 = test_p**2 / 2 * p0
Q = 1 - p2
S = lmbd_z_lin * Q
k = test_p * Q
table_vals[0, :] = [p0, p1, p2, Q, S, k]

for i in range(rows):
    T_req = generate_request_time_linear(tz_min, tz_max, work_time)
    T_processing = generate_process_time_linear(ts_min, ts_max, len(T_req))

    processed_requests, fot_times = simulate(T_req, T_processing, work_time)

    p0 = fot_times[0] / work_time
    p1 = fot_times[1] / work_time
    p2 = fot_times[2] / work_time

    Q = processed_requests / len(T_req)
    S = processed_requests / work_time
    k = (fot_times[1] * 1 + fot_times[2] * 2) / work_time

    table_vals[i + 1, :] = [p0, p1, p2, Q, S, k]

table_vals[-2,] = np.mean(table_vals[1:-2], axis=0)
table_vals[-1,] = np.std(table_vals[1:-2], axis=0)
df = pd.DataFrame(table_vals, columns=table_cols, index=indexes)

df

,p0,p1,p2,Q,S,k
theor,0.064000,0.288000,0.648000,0.352000,0.528000,1.584000
1,0.013507,0.213573,0.772920,0.391320,0.586111,1.759413
2,0.012958,0.213825,0.773217,0.395263,0.593333,1.760260
3,0.015255,0.216496,0.768249,0.392273,0.586667,1.752993
4,0.013526,0.209325,0.777149,0.392342,0.589167,1.763623
5,0.014129,0.211767,0.774105,0.398631,0.598611,1.759976
6,0.013173,0.207028,0.779799,0.392459,0.586944,1.766626
7,0.014167,0.213349,0.772484,0.393816,0.590833,1.758318
8,0.013396,0.208835,0.777769,0.387593,0.581389,1.764373
9,0.012980,0.213172,0.773848,0.390109,0.587222,1.760868


In [17]:
table_vals = np.zeros((rows + 3, 6))

test_p = lmbd_z_exp / lmbd_s_exp
p0 = 1 / (1 + test_p + test_p**2 / 2)
p1 = test_p * p0
p2 = test_p**2 / 2 * p0
Q = 1 - p2
S = lmbd_z_exp * Q
k = test_p * Q
table_vals[0, :] = [p0, p1, p2, Q, S, k]

for i in range(rows):
    T_req = generate_request_time_exp(lmbd_z_exp, work_time)
    T_processing = generate_process_time_exp(lmbd_s_exp, len(T_req))

    processed_requests, fot_times = simulate(T_req, T_processing, work_time)

    p0 = fot_times[0] / work_time
    p1 = fot_times[1] / work_time
    p2 = fot_times[2] / work_time

    Q = processed_requests / len(T_req)
    S = processed_requests / work_time
    k = (fot_times[1] * 1 + fot_times[2] * 2) / work_time

    table_vals[i + 1, :] = [p0, p1, p2, Q, S, k]

table_vals[-2,] = np.mean(table_vals[1:-2], axis=0)
table_vals[-1,] = np.std(table_vals[1:-2], axis=0)

pd.DataFrame(table_vals, columns=table_cols, index=indexes)

,p0,p1,p2,Q,S,k
theor,0.117647,0.352941,0.529412,0.470588,0.705882,1.411765
1,0.124016,0.358126,0.517858,0.473436,0.698056,1.393843
2,0.119078,0.361063,0.519859,0.470783,0.709444,1.400781
3,0.125913,0.347037,0.527050,0.475005,0.715278,1.401137
4,0.108178,0.355766,0.536055,0.475170,0.720278,1.427877
5,0.132043,0.367696,0.500260,0.491558,0.719722,1.368217
6,0.112273,0.353189,0.534537,0.472945,0.701667,1.422264
7,0.099214,0.355290,0.545496,0.465571,0.708056,1.446282
8,0.117114,0.350986,0.531900,0.462352,0.704444,1.414786
9,0.116022,0.349755,0.534223,0.464104,0.684167,1.418201
